In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from json import load

Primero carguemos los datos resultantes del proceso de ETL:

In [ ]:
tabla=pd.read_csv('ETL.csv')
tabla.head()

Ahora miremos datos nulos:

In [ ]:
tabla.isnull().sum()/33 # se divide sobre 33 para determinar la cantidad de pollos

Esto quiere decir que hay 5 pollos que en el archivo original no tienen la metadata como granja, edad, plan de vacunacion etc.

Encontremos cual es la fecha de estos registros y luego revisemos el archivo de Excel:

In [ ]:
tabla.loc[tabla.loc[pd.isna(tabla["ciclo"])].index]['fecha']

Luego de revisar Excel se puede llegar a la conclusion que en la semana 29 se tiene registro solo de 20 pollos. Todas las otras semanas siempre registran 25 pollos.

Para solucionar esto se pueden eliminar los registros que tengan estos campos vacios, algo que permitiria calcular la cantidad total de pollos registrados:

In [ ]:
tabla.dropna(subset=['ciclo'],inplace=True)
tabla.isnull().sum()

In [ ]:
print("Cantidad de pollos registrados:")
tabla['lesionTipo'].value_counts().sum()/33

Empecemos a revisar los datos erroneos de cada atributo.

Los atributos 'lesionTipo' y 'lesionRango' son generados a traves de un DataFrame en especifico para todos los pollos, no a partir del archivo de Excel directamente, asi que estas dos columnas no tienen datos erroneos.

**fecha:**

Si al convertir las fechas en el dtype 'datatime' no se genera ningun error, quiere decir que las fechas si tienen un buen formato. 

In [ ]:
pd.to_datetime(tabla['fecha'])

Asi que estos datos no son erroneos desde el punto de vista del formato. Si son erroneos por el hecho de que no fueron registrados ese dia, ya es algo fuera del alcance actual.

**ciclo:**

In [ ]:
tabla['ciclo'].value_counts()/33

Todos los resultados se repiten una cantidad considerable de veces como para asumir que ninguno es un dato erroneo, ademas que son numeros entre el 35 y el 39, ninguno es atipico.

**noGalpon:**

In [ ]:
np.sort(tabla['noGalpon'].unique())

Tranquilamente el numero de un galpon de pollos puede ser cualquier numero del 1 al 11.

**planVacuna,influenzaVacuna,newcastleVacuna,gumboroVacuna:**

In [ ]:
tabla['planVacuna'].value_counts()/33

In [ ]:
tabla['influenzaVacuna'].value_counts()/33

In [ ]:
tabla['newcastleVacuna'].value_counts()/33

In [ ]:
tabla['gumboroVacuna'].value_counts()/33

Ninguno de estos 4 atributos tampoco presenta datos erroneos. Pero se plantea una interrogante ¿Es necesario guardar newcastleVacuna sabiendo que siempre es la misma vacuna?

**nAnimal:**

In [ ]:
np.sort(tabla['nAnimal'].unique())

Tal cual se esperaba, los unicos numeros que deberian estar son {1,2,3,4,5}

**integridadIntestinal:**

In [ ]:
tabla['integridadIntestinal'].value_counts()/33

En este caso en ninguna de las semanas registradas en Excel se muestra el rango para este atributo, pero asumiendo por los tipos de datos que aunque el numero 6 y 5 apenas se registran solo un par de veces puede que tenga sentido si se evalua en una escala de 0 a 10. Asi que se asume que no son erroneos. Los registros con 0 son registros que estaban vacios en Excel.

In [ ]:
tabla['condicionHigado'].value_counts()/33

Segun el archivo de Excel, los score pueden ser TX-C-T asi que se asume que estas combinaciones son correctas, ya se podria consultar con algun experto para validar esto.

**edadEnDias:**

In [ ]:
np.sort(tabla['edadEnDias'].unique())

Cualquiera de estos numeros puede ser tranquilamente la edad de un pollo, asi que estos datos tambien estan correctos. 

**granja:**

In [ ]:
tabla['granja'].value_counts()/33

El archivo json arroja las siguientes granjas:

In [ ]:
load(open('aux_data.json'))[0]['granja']

Tecnicamente si todos los valores de la columna 'granja' se les aplica lower(), luego se remueven los espacios ' ' y se evalua cada uno de estos resultados como llaves del diccionario de granjas(obtenido del json) no se tendria ningun dato erroneo, pero si nos ponemos mas tecnicos, 15 pollos tienen granjas erroneas, 5 con 'Vergel ' en lugar de 'Vergel' y otras 10 tienen o 'Santa Defina' o 'Sta. Delfina' los cuales deberian ser 'Santa Delfina'. 

Igual solucionar este caso tampoco es dificil:

In [ ]:
tabla['granja'].str.strip()\
    .replace(['Sta. Delfina','Santa Defina'],'Santa Delfina')\
    .value_counts()/33

**sexoAnimales:**

In [ ]:
tabla['sexoAnimales'].value_counts()/33

Los correctos son M y H. Todos los datos erroneos a excepcion de 'S' y 'J' se pueden corregir de la siguiente manera:

In [ ]:
tabla['sexoAnimales'].replace(
    {
        'M ':'M',
        'Macho':'M',
        'Mac':'M',
        'Hem':'H',
        'Hembra':'H',
    }
).value_counts()/33

Quedarian 4 pollos con sexos erroneos. El simbolo '-' es para aquellos que no se les asignó nunca el atributo de sexo.

**bursometro:**
Este debe estar entre 0 y 8

In [ ]:
np.sort(tabla['bursometro'].unique())

Hay registros que tienen -1 o 9, busquemolos:

In [ ]:
tabla.query("bursometro>8 or bursometro<0").shape

El 9 pudo haber sido un error de tecleo, pero no se puede saber a ciencia cierta. El -1 si fue algo que se ingresó a proposito, pero tampoco se puede saber que se queria hacer. En este caso no se pueden modificar los datos erroneos. Ya se debe discutir si se mantienen estos datos o se eliminan de la tabla. 

**lesionTipo:**

Se asume que el dato es erroneo cuando no pertenece al rango de la lesion

In [ ]:
datos_lesiones=tabla[['lesionTipo','lesionRango']].value_counts().reset_index()[['lesionTipo','lesionRango']]
erroneos=[]
for lesion,rango in zip(datos_lesiones['lesionTipo'],datos_lesiones['lesionRango']):
    print(lesion,rango)
    conjunto=tabla.groupby('lesionTipo')['lesionPromedio'].unique()[lesion]
    print(conjunto)
    mayor=int(rango[-1])
    cantidad=tabla.query("lesionTipo==@lesion and (lesionPromedio>@mayor or lesionPromedio<0)").shape[0]
    if cantidad>0:
        erroneos.append({'lesion':lesion,'erroneos':cantidad})
    print()

A continuacion se muestran las lesiones que tienen datos erroneos con sus respectivas cantidades:

In [ ]:
erroneos

Al igual que bursometro no se puede saber a ciencia cierta que hacer con estos datos erroneos para modificarlos y que ya vuelvan datos correctos. La unica alternativa es discutir si mantener estos datos o eliminarlos de la tabla.

Por ultimo, calculemos la cantidad de registros y pollos con datos erroneos:

In [ ]:
tabla_e1=tabla.query("granja=='Vergel ' or granja=='Sta. Delfina' or granja=='Santa Defina'")

sexos=tabla['sexoAnimales'].unique()
query=f"sexoAnimales=='{sexos[3]}'"
for x in sexos[4:]:
    query+=f" or sexoAnimales=='{x}'"
tabla_e2=tabla.query(query)

tabla_e3=tabla.query("bursometro>8 or bursometro<0")

datos_lesiones=tabla[['lesionTipo','lesionRango']].value_counts().reset_index()[['lesionTipo','lesionRango']]
lista_e=[]
for lesion,rango in zip(datos_lesiones['lesionTipo'],datos_lesiones['lesionRango']):
    mayor=int(rango[-1])
    consulta=tabla.query("lesionTipo==@lesion and (lesionPromedio>@mayor or lesionPromedio<0)")
    if consulta.shape[0]>0:
        lista_e.append(consulta)
tabla_e4=pd.concat(lista_e)

In [ ]:
print("Cantidad de registros con al menos un dato erroneo:")
pd.concat([tabla_e1,tabla_e2,tabla_e3,tabla_e4]).drop_duplicates().shape[0]

In [ ]:
print("Pollos con datos erroneos:")
x=pd.concat([tabla_e1,tabla_e2,tabla_e3]).drop_duplicates().shape[0]
x/33+(pd.concat([tabla_e1,tabla_e2,tabla_e3,tabla_e4]).drop_duplicates().shape[0]-x)

# Preguntas:

¿Cuántos pollos tienen datos erróneos? 

R:\ 73

¿Qué hacemos con datos erróneos?

R:\ Hay casos donde pueden ser reemplazados por valores correctos, hay otros que se deben eliminar o mantener asi ya que no hay manera saber cual es el dato correcto. Esto igual se explica a lo largo del notebook.

¿Cuántos registros de la tabla final contienen datos erróneos?

R:\ 2281 tienen al menos un atributo erroneo.